In [1]:
import pandas as pd
import numpy as np

# Random seed for reproducibility
seed = 202
np.random.seed(seed)

# Import the train set and test set
train_data = pd.read_csv("polimi_dataset_challenge_train_v1.csv", delimiter=",")
test_data = pd.read_csv("polimi_dataset_challenge_test_to_predict_v1.csv", delimiter=",")
train_data2 = pd.read_csv("polimi_dataset_challenge_train_v1.csv", delimiter=",")
test_data2 = pd.read_csv("polimi_dataset_challenge_test_to_predict_v1.csv", delimiter=",")

In [2]:
def normalize_costumer_age(dataset):
    #Take all unique Cosumer_Age, assigns to each age alphabetically ordered a mean age
    customer_age_sort = sorted(dataset['CustomerAge'].dropna().unique())

    mapping = {}
    for idx, val in enumerate(customer_age_sort):
        mapping[val] = int(15+idx*10)
        
    ages = dataset['CustomerAge']
    ages.replace(mapping, inplace=True)

    #print(mapping, regions)
    ages = np.asarray(ages)
    ages = pd.DataFrame({'CustomerAge': ages})
    
     # Remove the previous CustomerAge and then insert the new CustomerAge
    dataset = dataset.drop('CustomerAge', axis=1)
    dataset = dataset.join(ages)
    return dataset

In [3]:
def normalize_region(dataset):
    #Take all unique Regions, assigns to each region alphabetically ordered a monotonic growing number
    regions_sort = sorted(dataset['Region'].dropna().unique())
    #regions_len = len(regions_sort) #20 Regions

    mapping = {}
    for idx, val in enumerate(regions_sort):
        mapping[val] = int(idx)

    regions = dataset['Region']
    regions.replace(mapping, inplace=True)

    #print(mapping, regions)
    regions = np.asarray(regions)
    regions = pd.DataFrame({'Region': regions})
    
    # Remove the previous Region columns and then insert the new Region
    dataset = dataset.drop('Region', axis=1)
    dataset = dataset.join(regions)
    return dataset

In [4]:
def normalize_province(dataset):
    #Take all unique Provinces, assigns to each province alphabetically ordered a monotonic growing number
    provinces_sort = sorted(dataset['Province'].dropna().unique())
    #provinces_len = len(provinces_sort) #110 Provinces

    mapping = {}
    for idx, val in enumerate(provinces_sort):
        mapping[val] = idx

    provinces = dataset['Province']
    provinces.replace(mapping, inplace=True)


    #print(provinces_sort, provinces_len)
    #print(mapping, provinces)
    provinces = np.asarray(provinces)
    provinces = pd.DataFrame({'Province': provinces})
    
    # Remove the previous Province columns and then insert the new Province
    dataset = dataset.drop('Province', axis=1)
    dataset = dataset.join(provinces)
    return dataset

In [5]:
def normalize_product(dataset):
    if('Product' in dataset.columns):
        #Take all unique Products, assigns to each product alphabetically ordered a monotonic growing number
        products_sort = sorted(dataset['Product'].dropna().unique())

        mapping = {}
        for idx, val in enumerate(products_sort):
            mapping[val] = int(idx)

        products = dataset['Product']
        products.replace(mapping, inplace=True)

        #print(mapping, products)
        products = np.asarray(products)
        products = pd.DataFrame({'Product': products})

        # Remove the previous Product columns and then insert the newProduct
        dataset = dataset.drop('Product', axis=1)
        dataset = dataset.join(products)
    return dataset

In [6]:
def drop_useless_columns(dataset):
    if('DataAllowanceOneShot' in dataset.columns and 'EstimatedDevicePrice' in dataset.columns):
        dataset = dataset.drop('DataAllowanceOneShot', axis=1)
        dataset = dataset.drop('EstimatedDevicePrice', axis=1)
    return dataset

In [7]:
def add_modified_sample(dataset):
    modified = []
    for i in dataset.isnull().any(axis=1):
        if(i):
            modified.append(1)
        else:
            modified.append(0)
    modified = np.asarray(modified)
    modified = pd.DataFrame({'IsModified': modified})
    dataset = dataset.join(modified)
    return dataset

In [8]:
def normalize_data_set(dataset):
    dataset = drop_useless_columns(dataset)
    dataset = add_modified_sample(dataset)
    dataset = normalize_costumer_age(dataset)
    dataset = normalize_region(dataset)
    dataset = normalize_province(dataset)
    dataset = normalize_product(dataset)
    return dataset

In [9]:
# Fill the missing values with the mean for each column
def fillna_mean_value(dataset):
    # fill missing values with mean column values
    dataset.fillna(dataset.mean(), inplace=True)
    return dataset

In [10]:
train_data = normalize_data_set(train_data)
test_data = normalize_data_set(test_data)
train_data.head()

,ID,DeviceFlag4G,DataArpu,DataAllowanceContinuous,DeviceFlagSmartphone,MonthlyVoiceTrafficCount,MonthlySMSTrafficCount,MonthlyDataTraffic,CustomerGender,CustomerExpatriate,...,Streaming-Applications,Terminals,Unclassified,VoIP,Web-Applications,IsModified,CustomerAge,Region,Province,Product
0,10930,1.0,0.156221,0.010514,1.0,0.018229,0.001623,0.011007,1.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0,45.0,16.0,14.0,0
1,10170,0.0,0.155086,0.054729,1.0,0.051432,0.001498,0.039382,0.0,0.0,...,0.071551,0.0,0.001896,0.000058,0.775410,1,25.0,NaN,NaN,0
2,1492,1.0,NaN,NaN,1.0,0.058594,0.001248,0.033031,1.0,1.0,...,0.099438,0.0,0.012055,0.003579,0.499158,1,35.0,NaN,NaN,0
3,7424,1.0,0.155086,0.004987,1.0,0.097005,0.000499,0.002928,0.0,0.0,...,0.041474,0.0,0.051692,0.013263,0.301528,0,55.0,8.0,15.0,2
4,4332,1.0,0.155086,0.038148,1.0,0.034505,0.000375,0.001002,1.0,0.0,...,0.005975,0.0,0.000857,0.019149,0.926208,0,65.0,19.0,103.0,0


In [11]:
train_data.to_csv('normalized_train_dataset.csv', index = False, encoding='utf-8')
test_data.to_csv('normalized_test_dataset.csv', index = False, encoding='utf-8')